# Simulating the Game

In [2]:
import numpy as np
import numpy.random as npr
from math import inf

def simulate_game(n):
    players = {i: (i+1)%n for i in range(n)}
    
    limit = inf
    
    p = 0
    
    while len(players) > 1:
        
        M = npr.uniform()
        
        if M < limit:
            p = players[p]
        else:
            for k, v in players.items():
                if v == p:
                    players[k] = players[p]
            
            p = players.pop(p)
            
        limit = min(M, limit)
    
    return p


def monte_carlo(N, winner, size):
    p = 0
    for _ in range(N):
        p += 1 if simulate_game(size) == winner else 0
        
    return p/N


In [3]:
N = int(1e7)
size, winner = 4, 3
print(f"p = {monte_carlo(N, winner, size)}")
#%timeit simulate_game()

p = 0.183407


# Estimating the Probability

We can show that the probability of $D$ winning the game will be given by:


$\begin{eqnarray} p_{D:ABCD} &=& \sum\limits_{a \ge 0} \sum\limits_{b \ge 0}\sum\limits_{c \ge 0} \dfrac{a(a+b+1)(a+b+c+2)}{(a+b+c+3)!} \cdot f(a,b,c) \\ &=& \sum\limits_{S \ge 0}\dfrac{S+2}{(S+3)!} \sum\limits_{k = 0}^{S} (k+1) \sum\limits_{a = 0}^{k} a \cdot f(a,k-a,S-k) \end{eqnarray}$

where $f(a,b,c)$ is a suitable mask that encodes $D$ as the winner. 

In [47]:
def f(a, b, c):
    a %= 4; b %= 3; c %= 2;
    return 1 if (a,b,c) in [(0,0,0), (0,1,1), (1,2,0), (1,0,1), (2,1,0), (2,2,1)] else 0


def calcula_soma(N):

    fat = 1.0/3 # (0+2)/(0+3)!

    soma = 0

    for S in range(N):
        soma += fat*sum((k+1)*sum(a*f(a,k-a,S-k) for a in range(k+1)) for k in range(S+1))
        fat *= (S+3.0)/((S+3)**2 - 1)

    print(soma)


In [49]:
calcula_soma(5000)
# 0.18343765086176353

0.18343765086176353


In [16]:
from math import cos, sin, exp, sqrt

hat = lambda i_c, i_s, w_c, w_s, e, c: (i_c*cos(1) + i_s*sin(1) + exp(-0.5)*(w_c*cos(sqrt(3)/2) + w_s*sqrt(3)*sin(sqrt(3)/2)) + e*exp(-1) + c)/24.0


In [56]:
args = ()
val = 2
dist = 1

for i_c in range(-20,21):
    if hat(i_c, -20, -20, -20, -50, 0) > soma + dist: 
        break
    for i_s in range(-20,21):
        if hat(i_c, i_s, -20, -20, -50, 0) > soma + dist: 
            break
        for w_c in range(-20,21):
            if hat(i_c, i_s, w_c, -20, -50, 0) > soma + dist: 
                break
            for w_s in range(-20,21):
                if hat(i_c, i_s, w_c, w_s, -50, 0) > soma + dist: 
                    break
                for e in range(-50,51):
                    curr = (i_c, i_s, w_c, w_s, e, 0)
                    if abs(hat(i_c, i_s, w_c, w_s, e, c) - soma) < abs(val - soma):
                        val = hat(i_c, i_s, w_c, w_s, e, c)
                        args = (i_c, i_s, w_c, w_s, e, c)
                        dist = abs(val - soma)
                    if hat(i_c, i_s, w_c, w_s, e, 0) > soma + dist: 
                        break


In [55]:
args

(-20, -8, -5, 17, 28, 0)